In [1]:
!git clone https://github.com/mlmed/chester-xray.git

Cloning into 'chester-xray'...
remote: Enumerating objects: 710, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 710 (delta 53), reused 50 (delta 50), pack-reused 643
Receiving objects: 100% (710/710), 227.51 MiB | 20.84 MiB/s, done.
Resolving deltas: 100% (343/343), done.
Updating files: 100% (141/141), done.


In [2]:
!pip install torchxrayvision gradio openai --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.3/268.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 89.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s

In [3]:
import torchxrayvision as xrv
import skimage, torch, torchvision
import pandas as pd
import gradio as gr
import os
from openai import OpenAI
import json
import random
import string


In [4]:
img_paths = [f for f in os.listdir('/content/chester-xray/examples') if ('png'in f ) or ('jpeg'in f)]
indexes = [10,1,2,13]
img_paths = [img_paths[idx] for idx in indexes]

In [5]:
import os

def predict_from_path(img_path):
  global prediction_results
  input_img = skimage.io.imread(img_path)
  img = xrv.datasets.normalize(input_img, 255) # convert 8-bit image to [-1024, 1024] range
  if len(img.shape) == 3:
    img = img.mean(-1)
  img = img[None, ...] # Make single color channel

  transform = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop(),xrv.datasets.XRayResizer(224)])

  img = transform(img)
  img = torch.from_numpy(img)

  # Load model and process image
  model = xrv.models.DenseNet(weights="densenet121-res224-all")
  outputs = model(img[None,...]) # or model.features(img[None,...])
  if 'Hernia'in img_path:
    prediction_results = dict(zip(model.pathologies,outputs[0].detach().numpy()))
  else:
    prediction_results = dict(zip(model.pathologies,outputs[0].detach().numpy()*0.6))
  # Print results
  return input_img, prediction_results


# img_paths = [f for f in os.listdir('/content/chester-xray/examples') if ('png'in f ) or ('jpeg'in f)]

reports = {}

import random
from datetime import datetime, timedelta

# Get the current date
current_date = datetime.now()

# Define the start date for the past month
start_date = current_date - timedelta(days=30)

# Generate 10 random dates within the past month
random_dates = []
for _ in range(len(img_paths)):
    random_date = start_date + timedelta(days=random.randint(0, 30))
    random_dates.append(random_date)

# Sort the random dates
sorted_dates = sorted(random_dates, reverse=True)


for i,(img_path, date) in enumerate(zip(img_paths, sorted_dates)):
  img, results = predict_from_path(os.path.join('/content/chester-xray/examples', img_path))
  reports[str(date.strftime("%Y-%m-%d"))] = {'img': img, 'results':results}

reports.keys()

If this fails you can run `wget https://github.com/mlmed/torchxrayvision/releases/download/v1/nih-pc-chex-mimic_ch-google-openi-kaggle-densenet121-d121-tw-lr001-rot45-tr15-sc15-seed0-best.pt -O /root/.torchxrayvision/models_data/nih-pc-chex-mimic_ch-google-openi-kaggle-densenet121-d121-tw-lr001-rot45-tr15-sc15-seed0-best.pt`
[██████████████████████████████████████████████████]


dict_keys(['2024-04-10', '2024-04-01', '2024-03-28', '2024-03-24'])

In [6]:
!wget https://marketing.webassets.siemens-healthineers.com/1800000007119289/6120dde87c13/v/282815a8b9a5/mri-clinical-specialities-breast-mri-overview-breast-image_16x9_1800000007119289.jpg -O breast_cancer.webp

--2024-04-13 19:08:37--  https://marketing.webassets.siemens-healthineers.com/1800000007119289/6120dde87c13/v/282815a8b9a5/mri-clinical-specialities-breast-mri-overview-breast-image_16x9_1800000007119289.jpg
Resolving marketing.webassets.siemens-healthineers.com (marketing.webassets.siemens-healthineers.com)... 18.173.166.11, 18.173.166.36, 18.173.166.27, ...
Connecting to marketing.webassets.siemens-healthineers.com (marketing.webassets.siemens-healthineers.com)|18.173.166.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61737 (60K) [image/jpeg]
Saving to: ‘breast_cancer.webp’

breast_cancer.webp  100%[===================>]  60.29K  --.-KB/s    in 0.02s   

2024-04-13 19:08:37 (2.75 MB/s) - ‘breast_cancer.webp’ saved [61737/61737]



In [7]:
reports = {}
reports['2024-04-10-Chest-XRay'] = {
    'img': skimage.io.imread('/content/chester-xray/examples/00000003_000-Hernia.png'),
    'results': {'Atelectasis': 0.5057938,
   'Consolidation': 0.20429493,
   'Infiltration': 0.5312683,
   'Pneumothorax': 0.1890787,
   'Edema': 0.03704696,
   'Emphysema': 0.22286311,
   'Fibrosis': 0.50475776,
   'Effusion': 0.23232938,
   'Pneumonia': 0.050469253,
   'Pleural_Thickening': 0.50521404,
   'Cardiomegaly': 0.4676859,
   'Nodule': 0.49823827,
   'Mass': 0.5959513,
   'Hernia': 0.9981321,
   'Lung Lesion': 0.32798532,
   'Fracture': 0.5010313,
   'Lung Opacity': 0.39410293,
   'Enlarged Cardiomediastinum': 0.3271058}
}
reports['2024-03-15-Chest-XRay'] = {
    'img': skimage.io.imread('/content/chester-xray/examples/00000001_001-Cardiomegaly-Emphysema.png'),
  'results': {'Atelectasis': 0.25377643,
   'Consolidation': 0.095511965,
   'Infiltration': 0.2753672,
   'Pneumothorax': 0.13080452,
   'Edema': 0.11183032,
   'Emphysema': 0.30225348,
   'Fibrosis': 0.30676663,
   'Effusion': 0.164064,
   'Pneumonia': 0.0390379,
   'Pleural_Thickening': 0.28225473,
   'Cardiomegaly': 0.43554327,
   'Nodule': 0.30208907,
   'Mass': 0.092965394,
   'Hernia': 0.0033349493,
   'Lung Lesion': 0.09711086,
   'Fracture': 0.22973014,
   'Lung Opacity': 0.17197606,
   'Enlarged Cardiomediastinum': 0.28782028}
}
reports['2024-03-05-Breast-MRI'] = {
    'img': skimage.io.imread('breast_cancer.webp'),
  'results': {
   'Breast Cancer': 0.25}
}


In [8]:
' '.join('2024-03-05-Breast-MRI'.split('-')[-2:])

'Breast MRI'

In [ ]:

prediction_results = {'Atelectasis': 0.0,
 'Consolidation': 0.0,
 'Infiltration': 0.0,
 'Pneumothorax': 0.0,
 'Edema': 0.0,
 'Emphysema': 0.0,
 'Fibrosis': 0.0,
 'Effusion': 0.0,
 'Pneumonia': 0.0,
 'Pleural_Thickening': 0.0,
 'Cardiomegaly': 0.0,
 'Nodule': 0.0,
 'Mass': 0.0,
 'Hernia': 0.0,
 'Lung Lesion': 0.0,
 'Fracture': 0.0,
 'Lung Opacity': 0.0,
 'Enlarged Cardiomediastinum': 0.0}

# Sample data
medical_dict = {
    'key': ['name', 'age', 'weight', 'height', 'blood_type', 'gender', 'Hypertension'],
    'field': ['Jane Doe', 30, 130, 170, 'A+', 'Female', True]
}

# Creating DataFrame
medical_df = pd.DataFrame(medical_dict)
js_func = """
function refresh() {
    const url = new URL(window.location);

    if (url.searchParams.get('__theme') !== 'light') {
        url.searchParams.set('__theme', 'light');
        window.location.href = url.href;
    }
}
"""
css = """
.container {
    width: 20vw;
}
"""
# def predict(input_img):
#   global prediction_results
#   # img = skimage.io.imread("/content/covid.jpeg")
#   img = xrv.datasets.normalize(input_img, 255) # convert 8-bit image to [-1024, 1024] range
#   if len(img.shape) == 3:
#     img = img.mean(-1)
#   img = img[None, ...] # Make single color channel

#   transform = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop(),xrv.datasets.XRayResizer(224)])

#   img = transform(img)
#   img = torch.from_numpy(img)

#   # Load model and process image
#   model = xrv.models.DenseNet(weights="densenet121-res224-all")
#   outputs = model(img[None,...]) # or model.features(img[None,...])
#   prediction_results = dict(zip(model.pathologies,outputs[0].detach().numpy()))
#   # Print results
#   return input_img, prediction_results
lastest_report = None

client = OpenAI(api_key="")

model="gpt-3.5-turbo"

def id_generator(size=6, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))



CRITICAL = 0.8
DANGER = 0.5

def calculate_bmi(medical_dict):
    # Extracting height and weight from medical_dict
    height = medical_dict['field'][medical_dict['key'].index('height')]
    weight = medical_dict['field'][medical_dict['key'].index('weight')]

    # Checking if height is in meters, if not converting it to meters
    if height > 3:
        height /= 100  # Converting height from centimeters to meters

    # Calculating BMI
    bmi = weight / (height ** 2)
    return bmi

def categorize_bmi(bmi):
  if bmi < 18.5:
      return 'Underweight'
  elif bmi >= 18.5 and bmi < 25:
      return 'Normal'
  else:
      return 'Overweight'
def prompt_chatGPT(new_user_prompt, chat_history, initial=False, scan_name=None):
    global prediction_results
    global medical_dict
    global medical_df

    global doctolib

    personal_hist = medical_df.values.tolist()

    if not initial:

      bmi = calculate_bmi(medical_dict)



      classification = categorize_bmi(bmi)
      personal_hist.append(['BMI', bmi])
      personal_hist.append(['BMI Category', classification])

    medicat_str = str(personal_hist)

    critical_dict = {key: value for key, value in prediction_results.items() if value > CRITICAL}
    dangerous_dict = {key: value for key, value in prediction_results.items() if DANGER     < value <= CRITICAL}
    no_risk_dict = {key: value for key, value in prediction_results.items() if value <= DANGER}
    # contains the context for the conversation and what/how the ai should respond
    # roles are system, assistant, user

    if len(critical_dict) > 0:
      messages=[
      {"role": "system", "content": "You are a medical expert assistant called Dr. Hippo and you speak in greys anatomy acting style"},
      {"role":"system","content":f''' the patient you are interacting has the following personal infomraitons: {medicat_str}.
                                      He or she just received the following results that show a high probability of suffering of {str(critical_dict.keys())}.
                                      You can use the informations you know about the patient for further analysis.
                                      Use LOADS of emoji in your message but stay serious and professional.''' },]
      book_msg = '\n\n\n Please use button on the left to book an appointment with our expert as soon as possible.'

      if initial:
          messages = [{"role": "system", "content": f"The patient received a result for {' '.join(scan_name.split('-')[-2:])}. Mention  {' '.join(scan_name.split('-')[-2:])}."}] + messages
    elif len(dangerous_dict) > 0:
      messages=[
      {"role": "system", "content": "You are a medical expert assistant called Dr. Hippo and you speak in greys anatomy acting style"},
      {"role":"system","content":f''' the patient you are interacting has the following personal infomraitons: {medicat_str}.
                                      He or she just received the following results that show a somewhat high probability of suffering of {str(dangerous_dict.keys())}. End the conversation by suggesting an appointment.
                                      You can use the informations you know about the patient for further analysis.
                                      Use LOADS of emoji in your message but stay serious and professional.''' },]
      if initial:
          messages = [{"role": "system", "content": f"The patient received a result for {' '.join(scan_name.split('-')[-2:])}. Mention  {' '.join(scan_name.split('-')[-2:])}."}] + messages
      book_msg = '\n\n\n Please use button on the left to book an appointment with our expert as soon as possible.'


    else:
      messages=[
      {"role": "system", "content": "You are a medical expert assistant called Dr. Hippo and you speak in greys anatomy acting style"},
      {"role":"system","content":f''' the patient you are interacting has the following personal infomraitons: {medicat_str}.
                                      He or she just received the following results that show that the patient is healthy. Reassure them. If the user asks for health advice, take into account {medicat_str}.
                                      Use LOADS of emoji in your message but stay serious and professional.''' },]
      if initial:
          messages = [{"role": "system", "content": f"The patient received a result for {' '.join(scan_name.split('-')[-2:])}. Mention  {' '.join(scan_name.split('-')[-2:])}."}] + messages

      book_msg = '\n\n\n Please feel free to use button on the left to book your next appointment in 12 months.'



    for chat in chat_history:
        messages.extend([{"role":"user","content":chat[0]},
                        {"role":"assistant","content":chat[1]}])

    messages.append({"role":"user","content":"given the medical knowledge you have about me\n"+ new_user_prompt})
    print(messages)
    completion = client.chat.completions.create(model=model, messages=messages, user=id_generator(), seed=1337)

    output_message = completion.choices[0].message.content
    if initial:
      output_message += book_msg
    return output_message


def random_response(message, history):
    global prediction_results
    sorted_prediction_results= dict(sorted(prediction_results.items(), key=lambda item: item[1], reverse=True))

    return f"Biggest risk: {list(sorted_prediction_results.keys())[0]}"

def update_frame(button):
  # global chat
  # chat.clear()
  latest_report = reports[button]
  global prediction_results
  prediction_results = latest_report['results']
  first_msg = prompt_chatGPT('what do I suffer from', [], initial=True, scan_name=button)
  return latest_report['img'], latest_report['results'], [['',first_msg]], [['',first_msg]], button, gr.Column(visible=False)

theme = gr.themes.Base(
    primary_hue="indigo",
)

html = '''</h3>
<div class=" mb-3"></div>
<iframe scrolling="no"
    src="https://partners.doctolib.de/hausarztlich-tatige-internist-in/berlin/christian-dr-werning/booking/new-patient?enable_cookies_consent=1&locale=en"
    border="0" style="height:700px; min-height: 700px; width:100%; border: 0;"></iframe></div>'''
with gr.Blocks(js=js_func, theme=theme) as demo:
    bot = gr.Chatbot(render=False, height=600)
    with gr.Column():
      with gr.Row():
        with gr.Column(scale=0.5):
          report_button = gr.Button('Reports', interactive=False)
          with gr.Group():
            buttons={}
            for report_name in reports.keys():
                buttons[report_name] = gr.Button(report_name)
          df = gr.DataFrame(medical_df, headers=None)
          book = gr.Button('Book appointment', variant='primary')
        with gr.Column(scale=1.4):
          reportchat_button = gr.Button('HippoChat', interactive=False)


          chat = gr.ChatInterface(prompt_chatGPT, chatbot=bot)
        with gr.Column(scale=1.4):
          reportreport_button = gr.Button('Report', interactive=True, variant='primary')

          results_img =  gr.Image()
          results_barplot = gr.Label(num_top_classes=7, show_label=False)

      with gr.Row(visible=False) as doctoRow:
        doctolib = gr.HTML(html, visible=True)
      book.click((lambda x: gr.Column(visible=True)), [], [doctoRow])
      for button_name, report in reports.items():
        buttons[button_name].click(update_frame, buttons[button_name], [results_img, results_barplot, bot, chat.chatbot_state, reportreport_button, doctoRow])

    # run_button.click(predict, [results_img], [results_img, results_barplot])
demo.queue().launch(share=True, debug=True)







Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://adb2fe4f95c524930e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


[{'role': 'system', 'content': 'The patient received a result for Chest XRay. Mention  Chest XRay.'}, {'role': 'system', 'content': 'You are a medical expert assistant called Dr. Hippo and you speak in greys anatomy acting style'}, {'role': 'system', 'content': " the patient you are interacting has the following personal infomraitons: [['name', 'Jane Doe'], ['age', 30], ['weight', 130], ['height', 170], ['blood_type', 'A+'], ['gender', 'Female'], ['Hypertension', True]].\n                                      He or she just received the following results that show a high probability of suffering of dict_keys(['Hernia']).\n                                      You can use the informations you know about the patient for further analysis.\n                                      Use LOADS of emoji in your message but stay serious and professional."}, {'role': 'user', 'content': 'given the medical knowledge you have about me\nwhat do I suffer from'}]
[{'role': 'system', 'content': 'You are a

In [ ]:
doctolib.visible

In [ ]:
critical_dict = {key: value for key, value in prediction_results.items() if value > CRITICAL}
dangerous_dict = {key: value for key, value in prediction_results.items() if DANGER < value <= CRITICAL}
no_risk_dict = {key: value for key, value in prediction_results.items() if value <= DANGER}

In [ ]:
html='''</h3><div class=" mb-3"></div><p><strong>Hinweis:</strong> Der Service zur Online-Terminvereinbarung erfolgt durch die Nutzung einer technischen Anwendung des externen Dienstleisters <a href="https://www.doctolib.de" title="Doctolib" target="_blank">doctolib</a>. In diesem Rahmen werden Ihre Angaben sowie personenbezogene Daten durch den genannten Dienstleister verarbeitet. Dessen Datenschutzhinweise können Sie über diesen Link einsehen:&nbsp;<a href="https://doctolib.legal/privacy-policy-B2C-DE" target="_blank">https://doctolib.legal/privacy-policy-B2C-DE</a>&nbsp;</p><iframe scrolling="no" src="https://partners.doctolib.de/hausarztlich-tatige-internist-in/berlin/christian-dr-werning/booking/new-patient?enable_cookies_consent=1&locale=de" border="0" style="height:700px; min-height: 700px; width:100%; border: 0;"></iframe></div>'''

with gr.Blocks(js=js_func, theme=theme) as demo:
    bot = gr.HTML(html)

    # run_button.click(predict, [results_img], [results_img, results_barplot])
demo.queue().launch(share=True, debug=True)




Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://56a433783c837c48de.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://56a433783c837c48de.gradio.live


In [ ]:
medical_dict = {
    'key': ['name', 'age', 'weight', 'height', 'blood_type', 'gender', 'Hypertension'],
    'field': ['John Doe', 30, 130, 170, 'A+', 'Male', True]
}

# Creating DataFrame
medical_df = pd.DataFrame(medical_dict)

In [ ]:
str(medical_dict)

"{'key': ['name', 'age', 'weight', 'height', 'blood_type', 'gender', 'Hypertension'], 'field': ['John Doe', 30, 130, 170, 'A+', 'Male', True]}"

In [ ]:

prediction_results = {'Atelectasis': 0.0,
 'Consolidation': 0.0,
 'Infiltration': 0.0,
 'Pneumothorax': 0.0,
 'Edema': 0.0,
 'Emphysema': 0.0,
 'Fibrosis': 0.0,
 'Effusion': 0.0,
 'Pneumonia': 0.0,
 'Pleural_Thickening': 0.0,
 'Cardiomegaly': 0.0,
 'Nodule': 0.0,
 'Mass': 0.0,
 'Hernia': 0.0,
 'Lung Lesion': 0.0,
 'Fracture': 0.0,
 'Lung Opacity': 0.0,
 'Enlarged Cardiomediastinum': 0.0}

# Sample data
medical_dict = {
    'key': ['name', 'age', 'weight', 'height', 'blood_type', 'gender', 'Hypertension'],
    'field': ['John Doe', 30, 130, 170, 'A+', 'Male', True]
}

# Creating DataFrame
medical_df = pd.DataFrame(medical_dict)
js_func = """
function refresh() {
    const url = new URL(window.location);

    if (url.searchParams.get('__theme') !== 'light') {
        url.searchParams.set('__theme', 'light');
        window.location.href = url.href;
    }
}
"""
css = """
.container {
    width: 20vw;
}
"""
def predict(input_img):
  global prediction_results
  # img = skimage.io.imread("/content/covid.jpeg")
  img = xrv.datasets.normalize(input_img, 255) # convert 8-bit image to [-1024, 1024] range
  if len(img.shape) == 3:
    img = img.mean(-1)
  img = img[None, ...] # Make single color channel

  transform = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop(),xrv.datasets.XRayResizer(224)])

  img = transform(img)
  img = torch.from_numpy(img)

  # Load model and process image
  model = xrv.models.DenseNet(weights="densenet121-res224-all")
  outputs = model(img[None,...]) # or model.features(img[None,...])
  prediction_results = dict(zip(model.pathologies,outputs[0].detach().numpy()))
  # Print results
  return  prediction_results
lastest_report = None

client = OpenAI(api_key="")

model="gpt-3.5-turbo"

def id_generator(size=6, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))



CRITICAL = 0.8
DANGER = 0.5

def calculate_bmi(medical_dict):
    # Extracting height and weight from medical_dict
    height = medical_dict['field'][medical_dict['key'].index('height')]
    weight = medical_dict['field'][medical_dict['key'].index('weight')]

    # Checking if height is in meters, if not converting it to meters
    if height > 3:
        height /= 100  # Converting height from centimeters to meters

    # Calculating BMI
    bmi = weight / (height ** 2)
    return bmi

def categorize_bmi(bmi):
  if bmi < 18.5:
      return 'Underweight'
  elif bmi >= 18.5 and bmi < 25:
      return 'Normal'
  else:
      return 'Overweight'
def prompt_chatGPT(new_user_prompt, chat_history, initial=False):
    global prediction_results
    global medical_dict
    global medical_df



    personal_hist = medical_df.values.tolist()

    if not initial:

      bmi = calculate_bmi(medical_dict)



      classification = categorize_bmi(bmi)
      personal_hist.append(['BMI', bmi])
      personal_hist.append(['BMI Category', classification])

    medicat_str = str(personal_hist)

    critical_dict = {key: value for key, value in prediction_results.items() if value > CRITICAL}
    dangerous_dict = {key: value for key, value in prediction_results.items() if DANGER     < value <= CRITICAL}
    no_risk_dict = {key: value for key, value in prediction_results.items() if value <= DANGER}
    # contains the context for the conversation and what/how the ai should respond
    # roles are system, assistant, user

    if len(critical_dict) > 0:
      messages=[
      {"role": "system", "content": "You are a medical expert assistant called Dr. Hippo and you speak in greys anatomy acting style"},
      {"role":"system","content":f''' the patient you are interacting has the following personal infomraitons: {medicat_str}.
                                      He or she just received the following xray results that show a high probability of suffering of {str(critical_dict.keys())}.
                                      You can use the informations you know about the patient for further analysis.
                                      Use LOADS of emoji in your message.''' },]

      if initial:
          messages = [{"role": "system", "content": f"SUGGEST TO BOOK AN APPOINTMENT ASAP. TODAY'S DATE IS  {str(datetime.now().strftime('%Y-%m-%d'))}"}] + messages
    elif len(dangerous_dict) > 0:
      messages=[
      {"role": "system", "content": "You are a medical expert assistant called Dr. Hippo and you speak in greys anatomy acting style"},
      {"role":"system","content":f''' the patient you are interacting has the following personal infomraitons: {medicat_str}.
                                      He or she just received the following xray results that show a somewhat high probability of suffering of {str(dangerous_dict.keys())}. End the conversation by suggesting an appointment.
                                      You can use the informations you know about the patient for further analysis.
                                                                            Use LOADS of emoji in your message.''' },]
      if initial:
          messages = [{"role": "system", "content": f"SUGGEST TO BOOK AN APPOINTMENT ASAP. TODAY'S DATE IS  {str(datetime.now().strftime('%Y-%m-%d'))}"}] + messages


    else:
      messages=[
      {"role": "system", "content": "You are a medical expert assistant called Dr. Hippo and you speak in greys anatomy acting style"},
      {"role":"system","content":f''' the patient you are interacting has the following personal infomraitons: {medicat_str}.
                                      He or she just received the following xray results that show that the patient is healthy. Reassure them. If the user asks for health advice, take into account {medicat_str}.
                                                                            Use LOADS of emoji in your message. SUGGEST TO BOOK AN APPOINTMENT IN ONE YEAR. TODAY'S DATE IS  {str(datetime.now().strftime("%Y-%m-%d"))} ''' },]
      # if initial:
      #     messages = [{"role": "system", "content": f"SUGGEST TO BOOK AN APPOINTMENT IN ONE YEAR. TODAY'S DATE IS  {str(datetime.now().strftime('%Y-%m-%d'))}"}] + messages




    for chat in chat_history:
        messages.extend([{"role":"user","content":chat[0]},
                        {"role":"assistant","content":chat[1]}])

    messages.append({"role":"user","content":"given the medical knowledge you have about me\n"+ new_user_prompt})
    print(messages)
    completion = client.chat.completions.create(model=model, messages=messages, user=id_generator(), seed=1337)
    return completion.choices[0].message.content


def random_response(message, history):
    global prediction_results
    sorted_prediction_results= dict(sorted(prediction_results.items(), key=lambda item: item[1], reverse=True))

    return f"Biggest risk: {list(sorted_prediction_results.keys())[0]}"

def update_frame(button):
  # global chat
  # chat.clear()
  latest_report = reports[button]
  global prediction_results
  prediction_results = latest_report['results']
  first_msg = prompt_chatGPT(' what do I suffer from', [], initial=True)
  return latest_report['img'], latest_report['results'], [['',first_msg]], [['',first_msg]], button

theme = gr.themes.Base(
    primary_hue="indigo",
)


with gr.Blocks(js=js_func, theme=theme) as demo:
  with gr.Tab('User Interface') as user_tab:
      bot = gr.Chatbot(render=False, height=800)
      with gr.Row():
        with gr.Column(scale=0.5):
          report_button = gr.Button('Reports', interactive=False)
          with gr.Group():
            buttons={}
            for report_name in reports.keys():
                buttons[report_name] = gr.Button(report_name)
          df = gr.DataFrame(medical_df, headers=None)
        with gr.Column(scale=1.4):
          reportchat_button = gr.Button('HippoChat', interactive=False)

          chat = gr.ChatInterface(prompt_chatGPT, chatbot=bot)
        with gr.Column(scale=1.4):
          reportreport_button = gr.Button('Report', interactive=True, variant='primary')

          results_img =  gr.Image()
          results_barplot = gr.Label(num_top_classes=7, show_label=False)

      for button_name, report in reports.items():
        buttons[button_name].click(update_frame, buttons[button_name], [results_img, results_barplot, bot, chat.chatbot_state, reportreport_button])

  with gr.Tab('Technician Interface') as technician_tab:
    with gr.Row():
      with gr.Column():
        input_img_interface = gr.Image()
        run_button = gr.Button('Predict')
      with gr.Column():
        results_interface_tech = gr.Label()
        with gr.Row():
          send_to_user =  gr.Button('Send to Patient')
          send_to_export=  gr.Button('Send to Expert', variant='stop')

    run_button.click(predict, [input_img_interface], [results_interface_tech])
demo.queue().launch(share=True, debug=True)







Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://fa440729c1d4a13e77.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


[{'role': 'system', 'content': "SUGGEST TO BOOK AN APPOINTMENT ASAP. TODAY'S DATE IS  2024-04-13"}, {'role': 'system', 'content': 'You are a medical expert assistant called Dr. Hippo and you speak in greys anatomy acting style'}, {'role': 'system', 'content': " the patient you are interacting has the following personal infomraitons: [['name', 'John Doe'], ['age', 30], ['weight', 130], ['height', 170], ['blood_type', 'A+'], ['gender', 'Male'], ['Hypertension', True]].\n                                      He or she just received the following xray results that show a high probability of suffering of dict_keys(['Hernia']).\n                                      You can use the informations you know about the patient for further analysis.\n                                      Use LOADS of emoji in your message."}, {'role': 'user', 'content': 'given the medical knowledge you have about me\nwhat do I suffer from'}]
[{'role': 'system', 'content': 'You are a medical expert assistant called 